# 6. Influence Analysis

This notebook analyzes the impact of the new data and logic, including the junior-senior relationships and the SSMS election results.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.max_rows = 10

## Load Datasets

In [ ]:
STUDENTS_PATH = "../data/students.csv"
EDGES_PATH = "../data/edges.csv"

try:
    students_df = pd.read_csv(STUDENTS_PATH)
    edges_df = pd.read_csv(EDGES_PATH)
    print("Successfully loaded datasets.")
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure you have run the data generation script first.")

## Construct the Graph

In [ ]:
G = nx.Graph()

# Add nodes with attributes from the students dataframe
for index, row in students_df.iterrows():
    node_id = row['id']
    attributes = row.to_dict()
    G.add_node(node_id, **attributes)

# Add edges with attributes from the edges dataframe
for index, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'], layer=row['layer'], weight=row['weight'])

## Simple Simulation

In [ ]:
from src.model import ElectionModel

model = ElectionModel(G.copy())
model.release_manifestos()

opinions_over_time = []
turnout_over_time = []

for i in range(10):
    model.step()
    
    # Collect data
    opinions = {post: {cand.id: [] for cand in cands} for post, cands in model.candidates_by_post.items()}
    turnout = []
    for agent in model.agents:
        for post, op in agent.opinion.items():
            for cand_id, score in op.items():
                opinions[post][cand_id].append(score)
        turnout.append(agent.turnout_propensity)
    
    avg_opinions = {post: {cand_id: np.mean(scores) for cand_id, scores in op.items()} for post, op in opinions.items()}
    avg_turnout = np.mean(turnout)
    
    opinions_over_time.append(avg_opinions)
    turnout_over_time.append(avg_turnout)

## Analysis

In [ ]:
# Compare the average opinion of juniors who have a senior connection to those who don't
juniors_with_seniors = [n for n, d in G.nodes(data=True) if d.get('batch') == 2024 and any([e[2]['layer'] == 'junior-senior' for e in G.edges(n, data=True)])]
juniors_without_seniors = [n for n, d in G.nodes(data=True) if d.get('batch') == 2024 and not any([e[2]['layer'] == 'junior-senior' for e in G.edges(n, data=True)])]

avg_opinion_with_seniors = {post: {cand.id: np.mean([model.grid.nodes[n]['agent'].opinion[post][cand.id] for n in juniors_with_seniors]) for cand in cands} for post, cands in model.candidates_by_post.items()}
avg_opinion_without_seniors = {post: {cand.id: np.mean([model.grid.nodes[n]['agent'].opinion[post][cand.id] for n in juniors_without_seniors]) for cand in cands} for post, cands in model.candidates_by_post.items()}

print("Average opinion of juniors with seniors:")
print(avg_opinion_with_seniors)
print("
Average opinion of juniors without seniors:")
print(avg_opinion_without_seniors)

# Compare the average turnout of SSMS winners to non-winners
ssms_winners = [n for n, d in G.nodes(data=True) if d.get('is_ssms_winner')]
non_winners = [n for n, d in G.nodes(data=True) if not d.get('is_ssms_winner')]

avg_turnout_ssms_winners = np.mean([model.grid.nodes[n]['agent'].turnout_propensity for n in ssms_winners])
avg_turnout_non_winners = np.mean([model.grid.nodes[n]['agent'].turnout_propensity for n in non_winners])

print(f"Average turnout of SSMS winners: {avg_turnout_ssms_winners}")
print(f"Average turnout of non-winners: {avg_turnout_non_winners}")